In [28]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

# Initialize the WebDriver (specify the path to your webdriver executable)
# Download the appropriate webdriver for your browser: https://selenium.dev/documentation/webdriver/
driver = webdriver.Chrome()

# Initialize lists to store data
names = []
postal_codes = []
categories = set()

# Loop through pages
for page in range(61):  # You mentioned pages from 0 to 60
    url = f'https://www.desocialekaart.be/zoeken?pagina={page}&paginaGrootte=25&rubriek=06.%20OPLEIDING,%20WERKLOOSHEID%20EN%20TEWERKSTELLING&sort=relevantie'
    driver.get(url)

    # Give the page some time to load (you might need to adjust the wait time)
    driver.implicitly_wait(5)  # Adjust the wait time as needed

    # Get the page source after it has loaded
    page_source = driver.page_source

    soup = BeautifulSoup(page_source, 'html.parser')
    organization_entries = soup.find_all('div', class_='row search-result')

    for entry in organization_entries:
        name = entry.find('h3').text.strip()
        postal_code = entry.find('div', class_='address').text.strip().split()[-1]
        category = entry.find('div', class_='taxonomies').text.strip()

        names.append(name)
        postal_codes.append(postal_code)
        categories.add(category)

# Close the WebDriver
driver.quit()

# Create a DataFrame
data = {
    'Name': names,
    'Postal Code': postal_codes
}
for category in categories:
    data[category] = [1 if category in entry else 0 for entry in categories]

df = pd.DataFrame(data)

# Save the data to an Excel file
df.to_excel('organizations_data.xlsx', index=False)
